In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

import keras 
from keras.models import Sequential,Model
from keras.layers import LSTM,Bidirectional,Dense,Input,Embedding,TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from nltk.tokenize import word_tokenize
import numpy as np

In [ ]:
from google.colab import drive,files
drive.mount("/content/drive")
dataM='/content/drive/My Drive/Colab Notebooks/MODELS/'
filepath = dataM+'BERT_CRF.ml'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
!pip install tf2crf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tf2crf import CRF 
from tf2crf import ModelWithCRFLoss

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
# data 
import pandas as pd
import json
path_to_json = '/content/drive/My Drive/Colab Notebooks/data/job_20k.json'
df  = pd.read_json(path_to_json,lines=True)
df

,clean_descrition,organization_list,skill_list,job_title_list
0,A technology services client of ours is lookin...,"[Swift, Strategic, Solutions, Inc]","[NoSQL, Teradata, Netezza, Cosmos, DB, JSON, A...","[Bigdata, Engineer]"
1,Job Title :Java Sr developerLocation: RemoteLo...,"[AQUA, Information, Systems,, Inc.]","[Java, JEE, J2EE]","[Java, developer]"
2,Mandatory Skills: Strong in Oracle database 19...,"[Softnet, Consulting, Inc]","[Oracle, database, 19c, Advanced, PL/SQL, prog...","[Oracle, PL/SQL, Lead, Architect]"
3,"Associate Scientist Thousand Oaks, CADescripti...","[Apex, Systems]","[Biochemistry, Biology, Analytical, Systems]","[Associate, Scientist]"
4,"Growing fintech company looking for Mid Level,...",[Jobot],"[Developer, JavaScript, SQL, ASP, HTML, Archit...","[Software, Engineer]"
...,...,...,...,...
19901,Tranzeal is an industry leading global Busines...,"[Tranzeal,, Inc.]","[Experience, with, mobile, software, developme...","[iOS, Developer]"
19902,About us: As a Fortune 50 company with more th...,"[Target, Corporation]",[Engineer],"[Engineer, -, US]"
19903,Ability to provide effective independent techn...,"[InfoVision,, Inc.]","[Informatica, or, Azure, Cloud, MDM, programs]","[MDM, Architect]"
19904,Requirements:5 years of experience as a . Net ...,"[Resourcesoft,, Inc.]","[.net, sitecore, cms, c#, web, services]","[Sitecore, Consultant]"


In [ ]:
df['clean_descrition'] = df['clean_descrition'].str.replace(r'[^\w\s]+', ' ')
df['clean_descrition'] = df['clean_descrition'].apply(lambda x: ' '.join(x.split()[:128]))
df['clean_descrition'][1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


'Job Title Java Sr developerLocation RemoteLong TermSkills java jeShould have good understanding on the change management role and had depth knowledge on the change advisory board process CAB Good knowledge on the incident management Problem management Change Management Should have involved in the 3L level of activities i e root cause analysis environment analysis understanding the application architecture infrastructure architecture The resource should have very good understanding on the SDLC process Very good knowledge on the Java J2EE and have hands on experience in understanding the architecture of heterogeneous application Good communication skills and team handling abilities Should have worked'

In [ ]:
jobs = list(df.clean_descrition)
def convert_to_list_then_dict(df_column= None, key_value=str):
    value_list = list(df_column)
    _ = list(set([item for value_list in value_list for item in value_list]))
    return dict.fromkeys(_, key_value)

org = convert_to_list_then_dict(df_column=df.organization_list, key_value='C')
skill = convert_to_list_then_dict(df_column=df.skill_list, key_value='S')
job = convert_to_list_then_dict(df_column=df.job_title_list, key_value='J')    

In [ ]:
# check text length
# for i in jobs:
#   if (len(i.split())) < 128:
#     jobs.remove(i)

In [ ]:
# for i in jobs:
#   if len(i.split()) !=128:
#     print(len(i.split()))

In [ ]:
# split
# XTr= jobs[:1500]
# XVal= jobs[1500:2000]
XTr= jobs[:151]
XVal= jobs[150:200]
org_b = org
skills_b = skill
jobs_b = job

In [ ]:
# XTr=['Google wants a backend developer with good coding skills','There is a position open for machine learning engineer, Meta is looking for people with machine learning background']
# XVal=['Amazon is looking for DevOps engineer']
# org_b={'google':'C','amazon':'C','meta':'C','microsoft':'C','qualcomm':'C'}
# skills_b={'coding':'S','machine':'S','learning':'S'}
# jobs_b={'backend':'J','developer':'J','engineer':'J','manager':'J','devops':'J'}

def make(X,org_b,skills_b,jobs_b):
  big=[]
  for sentence in X:
    token=word_tokenize(sentence)
    small=[]
    token_t=nltk.pos_tag(token)
    for f,s in token_t:
      tag='O'
      if f.lower() in org_b:
        tag=org_b[f.lower()]
      if f.lower() in skills_b:
        tag=skills_b[f.lower()]
      if f.lower() in jobs_b:
        tag=jobs_b[f.lower()]
      small.append((f,s,tag))
    big.append(small)
  return big
  
XTr_=make(XTr,org_b,skills_b,jobs_b)
XVal_=make(XVal,org_b,skills_b,jobs_b)

In [ ]:
msgLen=128

def fixMsg(X):
  global msgLen
  Xc=[]
  for i in range(len(X)):
    if len(X[i])<msgLen:
      while len(X[i])<msgLen:
        X[i].append(('UNK','UNK','UNK'))
    elif len(X[i])>msgLen:
      temp=X[i]
      del X[i]
      lb=0;chunk=msgLen
      while lb<len(temp):
        split=[]
        for j in range(lb,lb+chunk):
          if j<len(temp):
            split.append(temp[j])
          else:
            split.append(('UNK','UNK','UNK'))
        X.append(split)
        print(X)
        lb+=chunk
  return X 

XTr_t=fixMsg(XTr_)
XVal_t=fixMsg(XVal_)

In [ ]:
# ## remove
# for x in XTr_t:
#   if len(x)!=msgLen:
#     # print((x))
#     # print(len(x))
#     XTr_t.remove(x)

In [ ]:
for x in XTr_t:
  # print(len(x))
  assert(len(x)==msgLen)
# print(XTr_t)
for x in XVal_t:
  # print(len(x))
  assert(len(x)==msgLen)
# print(XVal_t)

In [ ]:
dict_={}
dict_1={}
token=1;token1=0
for x in XTr_t:
  for a,b,c in x:
    if a not in dict_:
      dict_[a]=token
      token=token+1
    if c not in dict_1:
      if c=='K':
        print(a,b,c)
      dict_1[c]=token1
      token1=token1+1
for x in XVal_t:
  for a,b,c in x:
    if a not in dict_:
      dict_[a]=token
      token=token+1
    if c not in dict_1:
      dict_1[c]=token1
      token1=token1+1
print('No of tokens words[{}] labels[{}]'.format(token,token1))

No of tokens words[4464] labels[5]


In [ ]:
XTr_fin=np.array(XTr)
XVal_fin=np.array(XVal)

yTr=np.array([[dict_1[element[2]] for element in sentence] for sentence in XTr_t])
yVal=np.array([[dict_1[element[2]] for element in sentence] for sentence in XVal_t])

In [ ]:
import tensorflow_text as text
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")

In [ ]:
text_preprocessed = preprocessor(XTr_fin)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (151, 128)
Word Ids   : [  101   138  2815  1826  7230  1104 17079  1110  1702  1370  2562 27922]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4
# https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3

bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4', trainable=False)

In [ ]:
input=Input(shape=(), dtype=tf.string)
model0=preprocessor(input)
seq_op=bert_layer(model0)['sequence_output']
model0=Model(input,seq_op)
model0.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 keras_layer_16 (KerasLayer)    {'input_type_ids':   0           ['input_17[0][0]']               
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                   

In [ ]:
XTr_fin=model0.predict(XTr_fin)
XVal_fin=model0.predict(XVal_fin)

2/2 [==============================] - 15s 5s/step


In [ ]:
print(XTr_fin.shape,XVal_fin.shape,type(XTr_fin),type(XVal_fin))

(151, 128, 768) (50, 128, 768) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
print(yTr.shape,yVal.shape,type(yTr),type(yVal))

(152, 128) (50, 128) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
input=Input(shape=(XTr_fin.shape[1],XTr_fin.shape[2]))
output=CRF(units=token1)(input)
mo=Model(input,output)
mo.summary()

model1=ModelWithCRFLoss(mo,sparse_target=True)
model1.compile(optimizer ='rmsprop')

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 128, 768)]        0         
                                                                 
 crf_8 (CRF)                 ((None, 128),             3870      
                              (None, 128, 5),                    
                              (None,),                           
                              (5, 5))                            
                                                                 
Total params: 3,870
Trainable params: 3,870
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model1.fit(np.array(XTr_fin),np.array(yTr),epochs=10,validation_data=[XVal_fin,yVal],batch_size=1)

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (16,8)
plt.plot(history.history['val_loss_val'])
plt.plot(history.history['loss'])
plt.xlabel('Epochs ->')
plt.ylabel('Losses ->')
plt.title('Losses v/s epochs.')
plt.legend(['Val Loss','Train Loss'])
plt.show()

In [ ]:
dict_1R={}
for k,v in dict_1.items():
  dict_1R[v]=k

In [ ]:
labels_=model1.predict(XVal_fin)
i=0 # i-th example from val dataset.
cnt=0
print()
for x in XVal_[0]:
  print('{} {}'.format(x[0],dict_1R[labels_[i][cnt]]))
  cnt+=1
  if cnt>10:
    break

In [ ]:
dict_1

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

def labelIt(yVal):
  yVal_lis=[]
  for y1 in yVal:
    temp=[]
    for y2 in y1:
      temp.append(dict_1R[y2])
    yVal_lis.append(temp)
  return yVal_lis 

yVal_=labelIt(yVal)
yPred_=labelIt(labels_)
labls=['C','J','S','O','UNK']
metrics.flat_f1_score(yVal_,yPred_,average='weighted',labels=labls)

In [ ]:
pip install -U 'scikit-learn<0.24'

In [ ]:
print(metrics.flat_classification_report(yVal_,yPred_,labels=labls,digits=3))